In [ ]:
import numpy as np

class Grid:
    """Class defining a one-dimensional Cartesian grid"""
    
    def __init__(self, lx, ly, lz, ncv):
        """Constructor
            lx .... total length of domain in x-direction [m]
            ly .... total length of domain in y-direction [m]
            lz .... total length of domain in z-direction [m]
            ncv ... number of control volumes in domain
        """
        # Store the number of control volumes
        self._ncv = ncv
        
        # Calculate the control volume length
        dx = lx/float(ncv)
        
        # Calculate the face locations
        self._xf = np.array([i*dx for i in range(ncv+1)])
        
        # Calculate the cell centroid locations
        self._xP = np.array([self._xf[0]] + 
                            [0.5*(self._xf[i]+self._xf[i+1]) for i in range(ncv)] +
                            [self._xf[-1]])
        
        # Calculate face areas
        self._Af = ly*lz*np.ones(ncv+1)
        
        # Calculate the outer surface area for each cell
        self._Ao = (2.0*dx*ly + 2.0*dx*lz)*np.ones(ncv)
        
        # Calculate cell volumes
        self._vol = dx*ly*lz*np.ones(ncv)
        
    @property
    def ncv(self):
        """Number of control volumes in domain"""
        return self._ncv
    
    @property
    def xf(self):
        """Face location array"""
        return self._xf
    
    @property
    def xP(self):
        """Cell centroid array"""
        return self._xP
    
    @property
    def dx_WP(self):
        return self.xP[1:-1]-self.xP[0:-2]
        
    @property
    def dx_PE(self):
        return self.xP[2:]-self.xP[1:-1]
      
    @property
    def Af(self):
        """Face area array"""
        return self._Af

    @property
    def Aw(self):
        """West face area array"""
        return self._Af[0:-1]
    
    @property
    def Ae(self):
        """East face area array"""
        return self._Af[1:]
    
    @property
    def Ao(self):
        """Outer face area array"""
        return self._Ao
    
    @property
    def vol(self):
        """Cell volume array"""
        return self._vol

In [ ]:
class ScalarCoeffs:
    """Class defining the set of coefficients for a finite-volume discretization
       of a scalar partial differential equation.
    """
    
    def __init__(self, ncv):
        """Constructor
            ncv ... number of control volumes in domain
        """
        self._ncv = ncv
        self._aP = np.zeros(ncv)
        self._aW = np.zeros(ncv)
        self._aE = np.zeros(ncv)
        self._rP = np.zeros(ncv)
        
    def zero(self):
        """Function to zero the coefficient arrays"""
        self._aP.fill(0.0)
        self._aW.fill(0.0)
        self._aE.fill(0.0)
        self._rP.fill(0.0)
        
    def accumulate_aP(self, aP):
        """Function to accumulate values onto aP"""
        self._aP += aP
        
    def accumulate_aW(self, aW):
        """Function to accumulate values onto aW"""
        self._aW += aW

    def accumulate_aE(self, aE):
        """Function to accumulate values onto aE"""
        self._aE += aE
        
    def accumulate_rP(self, rP):
        """Function to accumulate values onto rP"""
        self._rP += rP
        
    @property
    def ncv(self):
        """Number of control volumes in domain"""
        return self._ncv
        
    @property
    def aP(self):
        """Cell coefficient"""
        return self._aP
    
    @property
    def aW(self):
        """West cell coefficient"""
        return self._aW
    
    @property
    def aE(self):
        """East cell coefficient"""
        return self._aE
    
    @property
    def rP(self):
        """Cell residual"""
        return self._rP

In [ ]:
from enum import Enum

class BoundaryLocation(Enum):
    """Enumeration class defining boundary condition locations"""
    WEST = 1
    EAST = 2

In [ ]:
class DirichletBc:
    """Class defining a Dirichlet boundary condition"""
    
    def __init__(self, phi, grid, value, loc):
        """Constructor
            phi ..... field variable array
            grid .... grid
            value ... boundary value
            loc ..... boundary location
        """
        self._phi = phi
        self._grid = grid
        self._value = value
        self._loc = loc
        
    def value(self):
        """Return the boundary condition value"""
        return self._value
    
    def coeff(self):
        """Return the linearization coefficient"""
        return 0
    
    def apply(self):
        """Applies the boundary condition in the referenced field variable array"""
        if self._loc is BoundaryLocation.WEST:
            self._phi[0] = self._value
        elif self._loc is BoundaryLocation.EAST:
            self._phi[-1] = self._value
        else:
            raise ValueError("Unknown boundary location")

In [ ]:
class NeumannBc:
    """Class defining a Neumann boundary condition"""
    
    def __init__(self, phi, grid, gradient, loc):
        """Constructor
            phi ........ field variable array
            grid ....... grid
            gradient ... gradient at cell adjacent to boundary
            loc ........ boundary location
        """
        self._phi = phi
        self._grid = grid
        self._gradient = gradient
        self._loc = loc
        
    def value(self):
        """Return the boundary condition value"""
        if self._loc is BoundaryLocation.WEST:
            return self._phi[1] - self._gradient*self._grid.dx_WP[0]
        elif self._loc is BoundaryLocation.EAST:
            return self._phi[-2] + self._gradient*self._grid.dx_PE[-1]
        else:
            raise ValueError("Unknown boundary location")
    
    def coeff(self):
        """Return the linearization coefficient"""
        return 1
    
    def apply(self):
        """Applies the boundary condition in the referenced field variable array"""
        if self._loc is BoundaryLocation.WEST:
            self._phi[0] = self._phi[1] - self._gradient*self._grid.dx_WP[0]
        elif self._loc is BoundaryLocation.EAST:
            self._phi[-1] = self._phi[-2] + self._gradient*self._grid.dx_PE[-1]
        else:
            raise ValueError("Unknown boundary location")

In [ ]:
class DiffusionModel:
    """Class defining a diffusion model"""
    
    def __init__(self, grid, phi, gamma, west_bc, east_bc):
        """Constructor"""
        self._grid = grid
        self._phi = phi
        self._gamma = gamma
        self._west_bc = west_bc
        self._east_bc = east_bc
        
    def add(self, coeffs):
        """Function to add diffusion terms to coefficient arrays"""
        
        # Calculate the west and east face diffusion flux terms for each face
        flux_w = - self._gamma*self._grid.Aw*(self._phi[1:-1]-self._phi[0:-2])/self._grid.dx_WP
        flux_e = - self._gamma*self._grid.Ae*(self._phi[2:]-self._phi[1:-1])/self._grid.dx_PE
        
        # Calculate the linearization coefficients
        coeffW = - self._gamma*self._grid.Aw/self._grid.dx_WP
        coeffE = - self._gamma*self._grid.Ae/self._grid.dx_PE
        coeffP = - coeffW - coeffE
        
        # Modify the linearization coefficients on the boundaries
        coeffP[0] += coeffW[0]*self._west_bc.coeff()
        coeffP[-1] += coeffE[-1]*self._east_bc.coeff()
        
        # Zero the boundary coefficients that are not used
        coeffW[0] = 0.0
        coeffE[-1] = 0.0
        
        # Calculate the net flux from each cell
        flux = flux_e - flux_w
        
        # Add to coefficient arrays
        coeffs.accumulate_aP(coeffP)
        coeffs.accumulate_aW(coeffW)
        coeffs.accumulate_aE(coeffE)
        coeffs.accumulate_rP(flux)
        
        # Return the modified coefficient array
        return coeffs
    

In [ ]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

def get_sparse_matrix(coeffs):
    """Function to return a sparse matrix representation of a set of scalar coefficients"""
    ncv = coeffs.ncv
    data = np.zeros(3*ncv-2)
    rows = np.zeros(3*ncv-2, dtype=int)
    cols = np.zeros(3*ncv-2, dtype=int)
    data[0] = coeffs.aP[0]
    rows[0] = 0
    cols[0] = 0
    if ncv > 1:
        data[1] = coeffs.aE[0]
        rows[1] = 0
        cols[1] = 1

    for i in range(ncv-2):
        data[3*i+2] = coeffs.aW[i+1]
        data[3*i+3] = coeffs.aP[i+1]
        data[3*i+4] = coeffs.aE[i+1]
        rows[3*i+2:3*i+5] = i+1
        cols[3*i+2] = i
        cols[3*i+3] = i+1
        cols[3*i+4] = i+2
        
    if ncv > 1:
        data[3*ncv-4] = coeffs.aW[-1]
        data[3*ncv-3] = coeffs.aP[-1]
        rows[3*ncv-4:3*ncv-2] = ncv-1
        cols[3*ncv-4] = ncv-2
        cols[3*ncv-3] = ncv-1
        
    return csr_matrix((data, (rows, cols)))

def solve(coeffs):
    """Function to solve the linear system and return the correction field"""
    # Get the sparse matrix
    A = get_sparse_matrix(coeffs)
    # Solve the linear system
    return spsolve(A, -coeffs.rP)

## Problem 1:

In [ ]:
#Problem 1
# For ncv 1
import numpy as np
from numpy.linalg import norm
# Problem1
# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 1
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 100
converged = 1e-6

# Define thermophysical properties
k = 60

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
'''
The temperature is fixed on both sides of the bar. 
Therefore, Dirichelet Boundary conditions were applied on both west and east boundary location
'''
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)   
east_bc = DirichletBc(T, grid, 273, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)
    #Saving the temperatures in an array for grid independence test
    T1 = T
    xP1 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1

plt.xlabel("x")
plt.ylabel("T")
plt.legend()
plt.show()

In [ ]:
#Problem 1
#Grid 2
# For ncv 2
import numpy as np
from numpy.linalg import norm
# Problem1
# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 2
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 100
converged = 1e-6

# Define thermophysical properties
k = 60

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
'''
The temperature is fixed on both sides of the bar. 
Therefore, Dirichelet Boundary conditions were applied on both west and east boundary location
'''
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)   
east_bc = DirichletBc(T, grid, 273, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)
    #Saving the temperatures in an array for grid independence test
    T2 = T
    xP2 = grid.xP

In [ ]:
#Problem 1
#Grid 3
# For ncv 4
import numpy as np
from numpy.linalg import norm
# Problem1
# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 4
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 100
converged = 1e-6

# Define thermophysical properties
k = 60

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
'''
The temperature is fixed on both sides of the bar. 
Therefore, Dirichelet Boundary conditions were applied on both west and east boundary location
'''
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)   
east_bc = DirichletBc(T, grid, 273, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)
    #Saving the temperatures in an array for grid independence test
    T3 = T
    xP3 = grid.xP

In [ ]:
#Problem 1
#Grid 4
# For ncv 8
import numpy as np
from numpy.linalg import norm
# Problem1
# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 8
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 100
converged = 1e-6

# Define thermophysical properties
k = 60

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
'''
The temperature is fixed on both sides of the bar. 
Therefore, Dirichelet Boundary conditions were applied on both west and east boundary location
'''
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)   
east_bc = DirichletBc(T, grid, 273, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)
    #Saving the temperatures in an array for grid independence test
    T4 = T
    xP4 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1

plt.xlabel("x")
plt.ylabel("T")
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(xP1, T1, label='ncv=1')
plt.plot(xP2, T2, label='ncv=2')
plt.plot(xP3, T3, label='ncv=4')
plt.plot(xP4, T4, label='ncv=8')
plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()
#Display the title
plt.title('Fig1: Temperature vs. Distance')
plt.show()


# Linear Heat Conduction
## Explanation of Result:

The Steady state heat conduction equation with heat generation (d/dt = 0 and egen = 0) is 
                      $$  d^2T/dx^2 = 0 $$
After Integrating twice, the temperature profile becomes:
                    $$    T = ax + b $$
               
where a and b are arbitrary constants.
                
As the equation of T is a linear, the temperature profile shown in Figure1 is also linear.
The temperature is different for east and west boundary, thus the temperature profile varies from 373 to 273.

## Grid Independence Test:

In figure1 shows all the temperature profiles for number of control volume(ncv) 1,2,4 and 8.
It is visible from the graph that there is no difference in the profiles even for 1 ncv. It can be concluded that the minimum number of contriol volume that can be used for this problem is 1. The problem converges only after one iteration as it is a linear heat conduction problem.



In [ ]:
class SurfaceConvectionModel:
    """Class defining a surface convection model"""
    
    def __init__(self, grid, T, ho, To):
        """Constructor"""
        self._grid = grid
        self._T = T
        self._ho = ho
        self._To = To
        
      
    def add(self, coeffs):
        """Function to add surface convection terms to coefficient arrays"""
        
        # Calculate the source term
        source = self._ho*self._grid.Ao*(self._T[1:-1] - self._To)
        
        # Calculate the linearization coefficients
        coeffP = self._ho*self._grid.Ao
        
        # Add to coefficient arrays
        coeffs.accumulate_aP(coeffP)
        coeffs.accumulate_rP(source)
        
        return coeffs
    

# Problem 2 (External Convection) :

In [ ]:
# Problem 2
# Grid Independence Study
# Grid 1 for ncv 4
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 4
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60

# Define convection parameters
ho = 12
To = 298

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added DiricheltBc at East and West Boundary Location at T = 373K
'''
The temperature is fixed on both side at 373K
'''
# Define boundary conditions
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)  #Temperature is fixed on both boundaries.
east_bc = DirichletBc(T, grid, 373, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
# Implemented Surface convection model for external convection
surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceConvection.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)

#Calculating the minimum temperature for Grid Indepedence test:
avg1 = np.mean(T)
print('Average Temp for ncv = 4 is ', avg1)

#Saving the solution for grid indepence test   
T1 = T
xP1 = grid.xP

In [ ]:
# Problem 2 
#Grid 2 for ncv 8
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 8
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60

# Define convection parameters
ho = 12
To = 298

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added DiricheltBc at East and West Boundary Location at T = 373K
'''
The temperature is fixed on both side at 373K
'''
# Define boundary conditions
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 373, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceConvection.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)

#Calculating the avg temperature for Grid Indepedence test:
avg2 = np.mean(T)
print('Average Temp for ncv = 8 is ', avg2)    
    
#Storing the solution for grid independence test   
T2 = T
xP2 = grid.xP

In [ ]:
# Problem 2
# Grid 3 for ncv = 16
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60

# Define convection parameters
ho = 12
To = 298

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added DiricheltBc at East and West Boundary Location at T = 373K
'''
The temperature is fixed on both side at 373K
'''
# Define boundary conditions
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 373, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceConvection.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)
    
#Calculating the avg temperature for Grid Indepedence test:
avg3 = np.mean(T)
print('Average Temp for ncv = 16 is ', avg3)


#Storing the solution fo grid indepence test
T3 = T
xP3 = grid.xP

In [ ]:
# Problem 2
# Grid 4 for ncv = 32
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 32
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60

# Define convection parameters
ho = 12
To = 298

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added DiricheltBc at East and West Boundary Location at T = 373K
'''
The temperature is fixed on both side at 373K
'''
# Define boundary conditions
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 373, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceConvection.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)
    
#Calculating the avg temperature for Grid Indepedence test:
avg4 = np.mean(T)
print('Average Temp for ncv = 32 is ', avg4)
    
#Storing the solution for grid indepence test
T4 = T
xP4 = grid.xP

In [ ]:
# Problem 2
# Grid 5 for ncv = 64
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 64
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60

# Define convection parameters
ho = 12
To = 298

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added DiricheltBc at East and West Boundary Location at T = 373K
'''
The temperature is fixed on both side at 373K
'''
# Define boundary conditions
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 373, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceConvection.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the avg temperature for Grid Indepedence test:
avg5 = np.mean(T)
print('Average Temp for ncv = 64 is ', avg5)
    
#Storing the solution for grid indepence test
T5 = T
xP5 = grid.xP

In [ ]:
# Problem 2
# Grid 6 for ncv = 84
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 84
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60

# Define convection parameters
ho = 12
To = 298

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 373

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added DiricheltBc at East and West Boundary Location at T = 373K
'''
The temperature is fixed on both side at 373K
'''
# Define boundary conditions
west_bc = DirichletBc(T, grid, 373, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 373, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface convection model
surfaceConvection = SurfaceConvectionModel(grid, T, ho, To)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceConvection.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the avg temperature for Grid Indepedence test:
avg6 = np.mean(T)
print('Average Temp for ncv = 84 is ', avg6)
    
#Storing the solution for grid indepence test
T6 = T
xP6 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1

plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()
plt.show()


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#Ploting all the curves for different ncv

plt.plot(xP1, T1, label='ncv=4')
plt.plot(xP2, T2, label='ncv=8')
plt.plot(xP3, T3, label='ncv=16')
plt.plot(xP4, T4, label='ncv=32')
plt.plot(xP5, T5, label='ncv=64')
plt.plot(xP6, T6, label='ncv=84')

plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()

#Display the title
plt.title('Fig2: Temperature profile for external convection\n')
plt.show()



from prettytable import PrettyTable
import random
    
x = PrettyTable()

x.field_names = ["ncv", "Average Temperature", "Difference in Temp"]

x.add_row([4, avg1, avg1])
x.add_row([8, avg2, -avg2+avg1])
x.add_row([16, avg3, -avg3+avg2])
x.add_row([32, avg4, -avg4+avg3])
x.add_row([64, avg5, -avg5+avg4])
x.add_row([84, avg6, -avg6+avg5])

print(x)

# Grid Independence test

Figure 2 shows the different curves for ncv 4,8,16,32 and 64. 
The table above shows the difference in temperature from its previous ncv.
It can be noted that the differnce is relatively lower for ncv 64 and 84.
For this problem ncv 64 can be selected as an appropriate grid.


In [ ]:
##Comparison between the analytial and numerical solution
%matplotlib inline
import matplotlib.pyplot as plt
import math
#Comparison between analytical and numerical solution
#Numerical Solution
i = 1
for T in T_solns:
    plt.plot(grid.xP, T, label= 'Numerical Solution')
    i += 1
#Analytical Solution
'''
m = sqrt(hp/kA)
h = convection coefficient = 12
k = thermal conductivity = 60
P = perimeter
Ac = Crosssectional Area
'''
m = 12.64 
L = 0.1
# 100 linearly spaced numbers
x = np.arange(0,0.101,.001)
#
R = ((1*(np.sinh(m*x))) + np.sinh(m*(L-x))) * (75 / np.sinh(m*L))
P = R + 298



plt.plot(x,P, 'g', label='analytical solution')
plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()
plt.title("Fig3: Numerical vs. Analytical Solution for external convection\n")
plt.show()

#Display the avg temp for analytical solution
Avg_temp = np.mean(P)
print("Average Temperature from the analytical solution is ", Avg_temp)

#Display the avg temp for the numerical solution
print("Average Temperature from the Numerical Solution is", avg5)



## Comparison between the analytical and numerical solution

Figure 3 shows the temperature profile for both analytical and numerical solutions. The difference between these two curves is not significant. As the temperature is fixed for both east and west boundary at 300K and the ambient temperature is 298K, the lowest temperature is in the middle making it perfectly symmetrical along the midplane.


# Problem 3 (Heat Generation) :

In [ ]:
class RobinBc:
    """Class defining a Robin boundary condition"""
    
    def __init__(self, phi, grid, T_inf, loc, ha, k):
        """Constructor
            phi ........ field variable array
            grid ....... grid
            T_inf........ Ambient Temperature
            ha is the convective heat transfer coefficient
            loc ........ boundary location
            k.......thermal conductivity
        """
        self._phi = phi
        self._grid = grid
        self._T_inf = T_inf
        self._ha = ha
        self._k = k
        self._loc = loc
        
    def value(self):
        """Return the boundary condition value"""
        if self._loc is BoundaryLocation.WEST:
            return (self._phi[1] + (self._grid.dx_WP[0]*(self._ha/self._k)*self._T_inf))/(1 + (self._grid.dx_WP[0]*(self._ha/self._k)))
        elif self._loc is BoundaryLocation.EAST:
            return ( self._phi[-2] + (self._grid.dx_PE[-1]*(self._ha/self._k)*self._T_inf))/(1 + (self._grid.dx_PE[-1]*(self._ha/self._k)))
        else:
            raise ValueError("Unknown boundary location")
    
    def coeff(self):
        """Return the linearization coefficient"""
        return 1 /(1 + (self._grid.dx_WP[0]*(self._ha/self._k)))
    
    def apply(self):
        """Applies the boundary condition in the referenced field variable array"""
        if self._loc is BoundaryLocation.WEST:
            self._phi[0] = (self._phi[1] + (self._grid.dx_WP[0]*(self._ha/self._k)*self._T_inf))/(1 + (self._grid.dx_WP[0]*(self._ha/self._k)))
        elif self._loc is BoundaryLocation.EAST:
            self._phi[-1] = ( self._phi[-2] + (self._grid.dx_PE[-1]*(self._ha/self._k)*self._T_inf))/(1 + (self._grid.dx_PE[-1]*(self._ha/self._k)))
        else:
            raise ValueError("Unknown boundary location")

In [ ]:
class HeatGenerationModel:
    """Class defining a surface convection model"""
    
    def __init__(self, grid, q):
        """Constructor"""
        self._grid = grid

        self._q = q #heat generated per volume
        
    def add(self, coeffs):
        """Function to add surface convection terms to coefficient arrays"""
        
        # Calculating the source term SpVp with a negative sign to subtract it.
        source =  - self._q*self._grid.vol
        #print(source)
        
        # Calculate the linearization coefficients
        coeffP = 0
        
        # Add to coefficient arrays
        coeffs.accumulate_aP(coeffP)
        coeffs.accumulate_rP(source)
        
        return coeffs

In [ ]:
#problem 3
#Grid 1 for ncv = 4
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 1
lz = 1
ncv = 4

grid = Grid(lx, ly, lz, ncv)


# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 26                            

# Define convection parameters
ha = 280
q = 50000

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 320

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added DiricheltBc at East and West Boundary Location at T = 373K
# Define boundary conditions
west_bc = RobinBc(T, grid, 323, BoundaryLocation.WEST, ha, k)    #Implementing RobinBc at both boundaries.
east_bc = RobinBc(T, grid, 313, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the Heat Generation model
HeatGeneration = HeatGenerationModel(grid, q)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = HeatGeneration.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)
    
#Calculating the minimum temperature for Grid Indepedence test:
min1 = np.min(T)
print('Minimum Temp for ncv = 4 is ', min1)
    
#Storing the solution for grid indepence test
T1 = T
xP1 = grid.xP

In [ ]:
#problem 3
#Grid 2 for ncv = 8
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 1
lz = 1
ncv = 8

grid = Grid(lx, ly, lz, ncv)


# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 26

# Define convection parameters
ha = 280
q = 50000

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 320

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added DiricheltBc at East and West Boundary Location at T = 373K
# Define boundary conditions
west_bc = RobinBc(T, grid, 323, BoundaryLocation.WEST, ha, k)
east_bc = RobinBc(T, grid, 313, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the Heat Generation model
HeatGeneration = HeatGenerationModel(grid, q)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = HeatGeneration.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)
    
#Calculating the minimum temperature for Grid Indepedence test:
min2 = np.min(T)
print('Minimum Temp for ncv = 8 is ', min2)
    
#Storing the solution for grid indepence test
T2 = T
xP2 = grid.xP

In [ ]:
#problem 3
#Grid 3 for ncv = 16
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 1
lz = 1
ncv = 16

grid = Grid(lx, ly, lz, ncv)


# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 26

# Define convection parameters
ha = 280
q = 50000

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 320

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added RobinBc at East and West Boundary Location at T = 373K
# Define boundary conditions
west_bc = RobinBc(T, grid, 323, BoundaryLocation.WEST, ha, k)
east_bc = RobinBc(T, grid, 313, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the Heat Generation model
HeatGeneration = HeatGenerationModel(grid, q)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = HeatGeneration.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    print(T)
    
#Calculating the minimum temperature for Grid Indepedence test:
min3 = np.min(T)
print('Minimum Temp for ncv = 16 is ', min3)
    
#Storing the solution for grid indepence test
T3 = T
xP3 = grid.xP

In [ ]:
#problem 3
#Grid 4 for ncv = 32
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 1
lz = 1
ncv = 32

grid = Grid(lx, ly, lz, ncv)


# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 26

# Define convection parameters
ha = 280
q = 50000

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 320

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Added DiricheltBc at East and West Boundary Location at T = 373K
# Define boundary conditions
west_bc = RobinBc(T, grid, 323, BoundaryLocation.WEST, ha, k)
east_bc = RobinBc(T, grid, 313, BoundaryLocation.EAST, ha, k)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the Heat Generation model
HeatGeneration = HeatGenerationModel(grid, q)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = HeatGeneration.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the avg temperature for Grid Indepedence test:
min4 = np.min(T)
print('Minimum Temp for ncv = 32 is ', min4)
    
#Storing the solution for grid indepence test
T4 = T
xP4 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1

plt.xlabel("x")
plt.ylabel("T")
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#Ploting all the curves for different ncv

plt.plot(xP1, T1, label='ncv=4')
plt.plot(xP2, T2, label='ncv=8')
plt.plot(xP3, T3, label='ncv=16')
plt.plot(xP4, T4, label='ncv=32')


plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()

#Display the title
plt.title('Fig4: Temperature profile for Heat Generation')
plt.show()



from prettytable import PrettyTable
import random
    
x = PrettyTable()

x.field_names = ["ncv", "Minimun Temperature", "Difference in Temp"]

x.add_row([4, min1, min1])
x.add_row([8, min2, -min2+min1])
x.add_row([16, min3, -min3+min2])
x.add_row([32, min4, -min4+min3])


print(x)

# Grid Independence Test:

Figure 4 displays the different curves generated for ncv 4,8, 16, and 32. From the table above, it can be seen that the difference between the minimum temperature for different ncv is extremely low. The results are not affected much for coarse to fine grid.
For this problem, the results generated for ncv=16 are used to compare with the analytical solution.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#Numerical Sotuion
i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label="Analytical Solution")
    i += 1

#Analytical solution
T1 = 323
T2 = 313
L = 0.05
x = np.arange(-0.05, 0.051, .001)
'''
P = Temperature distribution from Analytical Solution.
P = (q/2K)*(L**2 - x**2) + ((T1 - T2)/(1 + k/(h*L)))*(x/2*L) + (T1 + T2)/2 +(q*L/h)
here, q = heat generated per volume
k = thermal conductivity
h = Covection coefficient
'''
#Applying the simplified equation for analytical solution
P = ((12500/13)*(L**2 - x**2)) + ((T2-T1)*x*(7/2)) + (T1+T2)/2 + (125/14)
#print(P)
#x = np.arange(0, 0.1, .001)
plt.plot(x, P, 'g', label = 'analytical solution')    
minP = np.min(P)


plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()
plt.title('Fig 5: Numerical vs. Analytical solution for internal heat generation\n')
plt.show()

#Display the minimum temp for both analytical and numerical solution
print('Minimum Temperature from analytical solution is:' , minP)
print('Minimum Temperature from numerical solution is:' , min3)

# Conclusion from Result: 


The analytical and numerical solutions can be compared from figure 5. The analytical solution is generated for -0.05 to 0.05 from the x-axis. Thus the curve shifted 0.05 in the negative x-direction. However, The minimum temperature calculated from both solutions is similar. I conclusion,  the numerical solution matches the analytical solution perfectly.





# Problem 4: Explicit Implementation for Steel

In [ ]:
# Problem 4
# Surface Radiation model for Explicit Implementation
class SurfaceRadiationModel:
    """Class defining a surface Radiation model"""
    
    def __init__(self, grid, T, Tc, epi, sigma):
        """Constructor"""
        self._grid = grid
        self._T = T
        self._epi = epi
        self._Tc = Tc
        self._sigma = sigma
        
    def add(self, coeffs):
        """Function to add surface radiation terms to coefficient arrays"""
        
        # Calculate the source term
        source = -self._epi*self._sigma*(self._T[1:-1]**4 - self._Tc**4)*self._grid.Ao
        
        # Calculate the linearization coefficients
        # For explicit implimentation of the source term, the linearization coefficient becomes zero.
        coeffP = 0     
        
        # Add to coefficient arrays
        coeffs.accumulate_aP(coeffP)
        coeffs.accumulate_rP(source)
        
        return coeffs

In [ ]:
#Problem 4 for steel
#Grid 1 for ncv = 4
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 4
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60         #Thermal Conductivity of steel

# Define convection parameters
h = 0
epi = 1                  #emmisivity
sigma = 5.67e-8          #Stefan-Boltzmann constant
Tc = 0                   

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
# The temperature at east and west boundary is fixed to 400K and 0K.
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface radiation model
surfaceRadiation = SurfaceRadiationModel(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg1 = np.mean(T)
print('Average Temp for ncv = 4 is ', avg1)

#Saving the solution for grid indepence test   
T1 = T
xP1 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1

plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()
plt.show()

In [ ]:
#Problem 4 for steel
#Grid 2 for ncv = 8
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 8
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60         #Thermal Conductivity of steel

# Define convection parameters
h = 0
epi = 1                  #emmisivity
sigma = 5.67e-8          #Stefan-Boltzmann constant
Tc = 0                   

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
# The temperature at east and west boundary is fixed to 400K and 0K.
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface radiation model
surfaceRadiation = SurfaceRadiationModel(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg2 = np.mean(T)
print('Average Temp for ncv = 8 is ', avg2)

#Saving the solution for grid indepence test   
T2 = T
xP2 = grid.xP

In [ ]:
#Problem 4 for steel
#Grid 3 for ncv = 16
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60         #Thermal Conductivity of steel

# Define convection parameters
h = 0
epi = 1                  #emmisivity
sigma = 5.67e-8          #Stefan-Boltzmann constant
Tc = 0                   

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
# The temperature at east and west boundary is fixed to 400K and 0K.
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface radiation model
surfaceRadiation = SurfaceRadiationModel(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg3 = np.mean(T)
print('Average Temp for ncv = 16 is ', avg3)

#Saving the solution for grid indepence test   
T3 = T
xP3 = grid.xP

In [ ]:
#Problem 4 for steel
#Grid 4 for ncv = 32
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 32
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60         #Thermal Conductivity of steel

# Define convection parameters
h = 0
epi = 1                  #emmisivity
sigma = 5.67e-8          #Stefan-Boltzmann constant
Tc = 0                   

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
# The temperature at east and west boundary is fixed to 400K and 0K.
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface radiation model
surfaceRadiation = SurfaceRadiationModel(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg4 = np.mean(T)
print('Average Temp for ncv = 32 is ', avg4)

#Saving the solution for grid indepence test   
T4 = T
xP4 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#Ploting all the curves for different ncv

plt.plot(xP1, T1, label='ncv=4')
plt.plot(xP2, T2, label='ncv=8')
plt.plot(xP3, T3, label='ncv=16')
plt.plot(xP4, T4, label='ncv=32')


plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()

#Display the title
plt.title('Fig 6: Temperature profile of Steel with surface Radiation (explicit)\n')
plt.show()


# Display a table showing the difference between average temperature
from prettytable import PrettyTable
import random
    
x = PrettyTable()

x.field_names = ["ncv", "Average Temperature", "Difference in Temp"]

x.add_row([4, avg1, avg1])
x.add_row([8, avg2, -avg2+avg1])
x.add_row([16, avg3, -avg3+avg2])
x.add_row([32, avg4, -avg4+avg3])


print(x)

## Grid Independence Test (Steel) :

Firgure 6 shows all the linear temparature profile for ncv 4,8,16 and 32. The table above shows that the difference in temprture is very low. thus using a grid with ncv=4 will be appropriate for this problem. The solution converges after 5 iteration.

# Problem 4: Implicit Implementation for Steel

In [ ]:
# Problem 4(2)
#Radiation model with implicit implementation 
class SurfaceRadiationModel2:
    """Class defining a surface convection model"""
    
    def __init__(self, grid, T, Tc, epi, sigma):
        """Constructor"""
        self._grid = grid
        self._T = T
        self._epi = epi
        self._Tc = Tc
        self._sigma = sigma
        
    def add(self, coeffs):
        """Function to add surface convection terms to coefficient arrays"""
        
        # Calculate the source term
        source = self._epi*self._sigma*(self._T[1:-1]**4 - self._Tc**4)*self._grid.Ao
        
        # Calculate the linearization coefficients
        coeffP = self._epi*self._sigma*((self._T[1:-1])**3)*4*self._grid.Ao      #Coefficients for implicit implementation
        
        # Add to coefficient arrays
        coeffs.accumulate_aP(coeffP)
        coeffs.accumulate_rP(source)
        
        return coeffs

In [ ]:
#Grid test 1 for ncv = 4
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 4
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60 #Thermal Conductivity of Steel

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface Radiation model
surfaceRadiation = SurfaceRadiationModel2(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg1 = np.mean(T)
print('Average Temp for ncv = 4 is ', avg1)

#Saving the solution for grid indepence test   
T1 = T
xP1 = grid.xP

In [ ]:
#Grid test 2 for ncv = 8
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 8
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60 #Thermal Conductivity of Steel

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface Radiation model
surfaceRadiation = SurfaceRadiationModel2(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg2 = np.mean(T)
print('Average Temp for ncv = 8 is ', avg2)

#Saving the solution for grid indepence test   
T2 = T
xP2 = grid.xP

In [ ]:
#Grid test 3 for ncv = 16
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 60 #Thermal Conductivity of Steel

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface Radiation model
surfaceRadiation = SurfaceRadiationModel2(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg3 = np.mean(T)
print('Average Temp for ncv = 16 is ', avg3)

#Saving the solution for grid indepence test   
T3 = T
xP3 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#Ploting all the curves for different ncv

plt.plot(xP1, T1, label='ncv=4')
plt.plot(xP2, T2, label='ncv=8')
plt.plot(xP3, T3, label='ncv=16')



plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()

#Display the title
plt.title('Fig 7: Temperature profile of Steel with surface Radiation (implicit)\n')
plt.show()


#Display a table showing the difference between average temperatures
from prettytable import PrettyTable
import random
    
x = PrettyTable()

x.field_names = ["ncv", "Average Temperature", "Difference in Temp"]

x.add_row([4, avg1, avg1])
x.add_row([8, avg2, -avg2+avg1])
x.add_row([16, avg3, -avg3+avg2])



print(x)

# Grid test for Steel (implicit)

Figure 7 shows all the curves for ncv 4,8 and 16. The difference is not significant. For this implicit method, ncv 32 has been taken for further comparison.

## Conclusion from Result (Steel):

From figure 6 and 7, we can see that the surface radiation problem converges for both implicit and explicit implementation of the source term. Because of the high thermal conductivity of steel (k = 60), the heat transfer does not depened much on the linearization method. Therefore, the surface radiation problem converges just after 3-5 iterations for both methods.


# Problem 4: Surface Radiation for Wood (implicit)

In [ ]:
#Problem 4: Implicit implementation for wood
#Grid test 1 for ncv = 4
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 4
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 0.1 #Thermal Conductivity of Wood

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface Radiation model
surfaceRadiation = SurfaceRadiationModel2(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg1 = np.mean(T)
print('Average Temp for ncv = 4 is ', avg1)

#Saving the solution for grid indepence test   
T1 = T
xP1 = grid.xP

In [ ]:
#Grid test 2 for ncv = 8
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 8
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 0.1 #Thermal Conductivity of wood

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface Radiation model
surfaceRadiation = SurfaceRadiationModel2(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg2 = np.mean(T)
print('Average Temp for ncv = 8 is ', avg2)

#Saving the solution for grid indepence test   
T2 = T
xP2 = grid.xP

In [ ]:
#Grid test 3 for ncv = 16
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 16
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 0.1 #Thermal Conductivity of Wood

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface radiation model
surfaceRadiation = SurfaceRadiationModel2(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg3 = np.mean(T)
print('Average Temp for ncv = 16 is ', avg3)

#Saving the solution for grid indepence test   
T3 = T
xP3 = grid.xP

In [ ]:
#Grid test 4 for ncv = 32
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 32
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 0.1 #Thermal Conductivity of Wood

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface radiation model
surfaceRadiation = SurfaceRadiationModel2(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg4 = np.mean(T)
print('Average Temp for ncv = 32 is ', avg4)

#Saving the solution for grid indepence test   
T4 = T
xP4 = grid.xP

In [ ]:
#Grid test 5 for ncv = 64
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 64
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 0.1 #Thermal Conductivity of Wood

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface radiation model
surfaceRadiation = SurfaceRadiationModel2(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg5 = np.mean(T)
print('Average Temp for ncv = 64 is ', avg5)

#Saving the solution for grid indepence test   
T5 = T
xP5 = grid.xP

In [ ]:
#Grid test 84 for ncv = 84
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 84
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 0.1 #Thermal Conductivity of Wood

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface radiation model
surfaceRadiation = SurfaceRadiationModel2(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
    
#Calculating the average temperature for Grid Indepedence test:
avg6 = np.mean(T)
print('Average Temp for ncv = 84 is ', avg6)

#Saving the solution for grid indepence test   
T6 = T
xP6 = grid.xP

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1

plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#Ploting all the curves for different ncv

plt.plot(xP1, T1, label='ncv=4')
plt.plot(xP2, T2, label='ncv=8')
plt.plot(xP3, T3, label='ncv=16')
plt.plot(xP4, T4, label='ncv=32')
plt.plot(xP5, T5, label='ncv=64')
plt.plot(xP6, T6, label='ncv=84')


plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()

#Display the title
plt.title('Fig 7: Temperature profile of Wood with surface Radiation (implicit)\n')
plt.show()


#Display a table showing the difference between average temperature
from prettytable import PrettyTable
import random
    
x = PrettyTable()

x.field_names = ["ncv", "Average Temperature", "Difference in Temp"]

x.add_row([4, avg1, avg1])
x.add_row([8, avg2, -avg2+avg1])
x.add_row([16, avg3, -avg3+avg2])
x.add_row([32, avg4, -avg4+avg3])
x.add_row([64, avg5, -avg5+avg4])
x.add_row([84, avg6, -avg6+avg5])

print(x)

# Grid Independence test (Wood) :

Figure 7 shows all the different curves obtained for ncv 4,8,16,32,64 and 84. It is visible from the table that, the temperature difference is relatively higher for the lower number of a control volume. For ncv 84 the difference is 0.67 thus it has been taken as an appropriate grid. For all 6 cases, the problem convereges after 6 iterations.

# Problem 4 : Explicit Implementation (Wood):

In [ ]:
#Problem 4: (Wood - Explicit)
# The grid used in implicit implementation has also been used for this one

#Grid test 6 for ncv = 84 
import numpy as np
from numpy.linalg import norm

# Define the grid
lx = 0.1
ly = 0.005
lz = 0.005
ncv = 84
grid = Grid(lx, ly, lz, ncv)

# Set the maximum number of iterations and convergence criterion
maxIter = 1000
converged = 1e-6

# Define thermophysical properties
k = 0.1 #Thermal Conductivity of Wood

# Define convection parameters
h = 0
epi = 1                 #Emmisivity
sigma = 5.67e-8         #Stefan-Boltzmann Constant
Tc = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

# Initial conditions
T0 = 300

# Initialize field variable arrays
T = T0*np.ones(grid.ncv+2)

# Define boundary conditions
west_bc = DirichletBc(T, grid, 400, BoundaryLocation.WEST)
east_bc = DirichletBc(T, grid, 0, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each iteration
T_solns = [np.copy(T)]

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

# Define the surface Radiation model 2 (Explicit Implementation of source term)
surfaceRadiation = SurfaceRadiationModel(grid, T, Tc, epi, sigma)

# Iterate until the solution is converged
for i in range(maxIter):
    # Zero the coefficients and add each influence
    coeffs.zero()
    coeffs = diffusion.add(coeffs)
    coeffs = surfaceRadiation.add(coeffs)

    # Compute residual and check for convergence 
    maxResid = norm(coeffs.rP, np.inf)
    avgResid = np.mean(np.absolute(coeffs.rP))
    print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
    if maxResid < converged:
        break
    
    # Solve the sparse matrix system
    dT = solve(coeffs)
    
    # Update the solution and boundary conditions
    T[1:-1] += dT
    west_bc.apply()
    east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    #print(T)
'''    
#Calculating the average temperature for Grid Indepedence test:
avg6 = np.mean(T)
print('Average Temp for ncv = 84 is ', avg6)

#Saving the solution for grid indepence test   
T6 = T
xP6 = grid.xP
'''

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1

plt.xlabel("Distance(x)")
plt.ylabel("Temperature(K)")
plt.legend()
plt.title("Fig 8 :Temperature profile for wood with explicit implementation\n")
plt.show()

## Comment: 

Figure 7 and 8 shows the temparute profile of Wood for implicit and explicit Implementation of the source term respectively. Here, the problem converges for implicit implementation but it does not converge for explicit implimentation of the source term.

# Conclusion: 

After reviewing all four cases of radiation heat transfer, we can conclude that for higher thermal conductivity (Steel) the heat transfer rate did not depend on the linearization method.
On the other hand, for a lower thermal conductivity (Wood), the linearization coefficient impacts the numerical procedure.  
